In [1]:
import pandas as pd
import numpy as np

from utils.metric_utils import HardMetric
from sklearn.metrics import f1_score

In [2]:
VALIDATION = False
SAMPLE = False
SAMPLE_FRAC = 0.1
final_metric = HardMetric(f1_score)

In [3]:
train = pd.read_csv('data/train.csv',index_col = 'id')
test = pd.read_csv('data/test.csv',index_col = 'id')
data = pd.concat([train, test]).drop(columns = 'target')
sample_test = pd.read_csv('data/sample_submission.csv',index_col = 'id')

C:\Alon\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
from sklearn.model_selection import train_test_split
if VALIDATION:
    if SAMPLE:
        data = train.sample(frac = SAMPLE_FRAC)
    else:
        data = train
    train, test = train_test_split(data, train_size=0.7, shuffle=True)
    y_test = test['target']
    test = test.drop(columns = 'target')

In [5]:
train.shape, test.shape

((7613, 4), (3263, 3))

# Feature Extraction

## Keyword

In [6]:
from feature_extraction.keyword import process_keyword

keyword = data['keyword']
X_keyword = process_keyword(keyword)
X_keyword.sample(10)

,keyword_OH_ablaze,keyword_OH_accident,keyword_OH_aftershock,keyword_OH_airplane%20accident,keyword_OH_ambulance,keyword_OH_annihilated,keyword_OH_annihilation,keyword_OH_apocalypse,keyword_OH_armageddon,keyword_OH_army,...,keyword_OH_whirlwind,keyword_OH_wild%20fires,keyword_OH_wildfire,keyword_OH_windstorm,keyword_OH_wounded,keyword_OH_wounds,keyword_OH_wreck,keyword_OH_wreckage,keyword_OH_wrecked,keyword_isna
id,,,,,,,,,,,,,,,,,,,,,
10756,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
7544,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8509,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3124,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8779,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3933,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
806,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7885,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9457,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Text

In [7]:
from feature_extraction.text import process_text

text = data['text']
X_text = process_text(text)
X_text.sample(10)

,text_our,text_deeds,text_are,text_the,text_reason,text_of,text_this,text_earthquake,text_may,text_allah,...,text_reserve,text_hattrick,text_ebolaoutbreak,text_ala,text_issuicide,text_rajman,text_hasaka,text_risen,text_fasteners,text_xrwn
id,,,,,,,,,,,,,,,,,,,,,
55,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
251,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10844,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8105,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7957,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8618,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4618,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5828,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## combine

In [8]:
X_all = X_keyword.join(X_text)

X_train = X_all.loc[train.index]
y_train = train['target']
X_test = X_all.loc[test.index]

# Modeling

In [9]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()

## fitting

In [10]:
clf.fit(X_train, y_train, )
y_pred_score = clf.predict_proba(X_test)[:, 1]

## Threshold choosing

In [11]:
from sklearn.model_selection import cross_val_predict

In [12]:
y_train_pred_score = cross_val_predict(clf, X_train, y_train, method = 'predict_proba')[:, 1]

In [13]:
y_pred = final_metric.get_thresh(y_train, y_train_pred_score)

# (Validation) Evaluation

In [14]:
if VALIDATION:
    from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, roc_curve, plot_roc_curve, plot_confusion_matrix, plot_precision_recall_curve
    import matplotlib.pyplot as plt
    print(f'total score:\t {final_metric(y_test, y_pred_score)}')
    plot_roc_curve(clf, X_test, y_test)
    plot_precision_recall_curve(clf, X_test, y_test)
    plot_confusion_matrix(clf, X_test, y_test)

# (Pred) Save Results

In [15]:
if not VALIDATION:
    import datetime
    
    current_time = str(datetime.datetime.now())[:-7].replace(' ', '__').replace('-', '_').replace(':', '_')
    pd.Series(data = y_pred, index = test.index, name='target').to_frame().to_csv(f'results/results_{current_time}.csv')